In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, "../scripts")
import helpers as h
import window_detect2 as w
import ml_window as m
from icecream import ic

import itertools


In [2]:
arrs = {}

# measured data
a00, a01 = h.import_desired_data("A", "15T")
arrs["meas"] = np.array(h.normalize(a01["Temp C"])) 
window_sched = np.array(a01["Window Open"])

# ambient data 
arrs["amb"] = np.array(h.normalize(a01["Ambient Temp"]))

# simulation data
sim_data_path = "../../data/energy_model/230307_and_smoothed.pkl"
sim_data = pd.read_pickle(sim_data_path)
arrs["sim"] = np.array(h.normalize(sim_data["Temp C"]))

In [3]:
# meausred/ambient temp difference 
diffs = {}
diffs["amb_minus_meas"] = arrs["amb"] - arrs["meas"]
diffs["meas_minus_amb"] = arrs["meas"] - arrs["amb"] 
diffs["abs_amb_meas_diff"] = np.abs(arrs["amb"] - arrs["meas"])

diffs = {k: h.normalize(v) for k,v in diffs.items()}

diff_keys = list(diffs.keys())

In [4]:
# derivaties
derivs = {}
derivs["dt_meas"] = np.gradient(arrs["meas"])
derivs["dt_sim"] = np.gradient(arrs["sim"])
derivs["dt_amb"] = np.gradient(arrs["amb"] )

derivs = {k: h.normalize(v) for k,v in derivs.items()}

# # derivatives of differences
# dt_amm = np.gradient(amb_minus_meas)
# dt_mma = np.gradient(meas_minus_amb)
# dt_aamd = np.gradient(abs_amb_meas_diff)

In [5]:
meas_keys = list(arrs.keys())
meas_set = meas_keys.copy()

# has to be the exact list of keys so dont duplicate existing 
for comb in itertools.combinations(meas_keys, 3):
    meas_set.append(comb)


meas_set

['meas', 'amb', 'sim', ('meas', 'amb', 'sim')]

In [6]:
deriv_keys = list(derivs.keys())
deriv_set = deriv_keys.copy()

# has to be the exact list of keys so dont duplicate existing 
for comb in itertools.combinations(deriv_keys, 3):
    deriv_set.append(comb)


deriv_set

['dt_meas', 'dt_sim', 'dt_amb', ('dt_meas', 'dt_sim', 'dt_amb')]

In [7]:
meas_and_deriv_keys = deriv_keys + meas_keys
meas_and_deriv_set = []

for comb in itertools.combinations(meas_and_deriv_keys, 2):
    meas_and_deriv_set.append(comb)
meas_and_deriv_set

[('dt_meas', 'dt_sim'),
 ('dt_meas', 'dt_amb'),
 ('dt_meas', 'meas'),
 ('dt_meas', 'amb'),
 ('dt_meas', 'sim'),
 ('dt_sim', 'dt_amb'),
 ('dt_sim', 'meas'),
 ('dt_sim', 'amb'),
 ('dt_sim', 'sim'),
 ('dt_amb', 'meas'),
 ('dt_amb', 'amb'),
 ('dt_amb', 'sim'),
 ('meas', 'amb'),
 ('meas', 'sim'),
 ('amb', 'sim')]

In [8]:
all_data = arrs | diffs | derivs
all_data.keys()

dict_keys(['meas', 'amb', 'sim', 'amb_minus_meas', 'meas_minus_amb', 'abs_amb_meas_diff', 'dt_meas', 'dt_sim', 'dt_amb'])

In [9]:
test_set = deriv_set + meas_set + meas_and_deriv_set + diff_keys
test_set

['dt_meas',
 'dt_sim',
 'dt_amb',
 ('dt_meas', 'dt_sim', 'dt_amb'),
 'meas',
 'amb',
 'sim',
 ('meas', 'amb', 'sim'),
 ('dt_meas', 'dt_sim'),
 ('dt_meas', 'dt_amb'),
 ('dt_meas', 'meas'),
 ('dt_meas', 'amb'),
 ('dt_meas', 'sim'),
 ('dt_sim', 'dt_amb'),
 ('dt_sim', 'meas'),
 ('dt_sim', 'amb'),
 ('dt_sim', 'sim'),
 ('dt_amb', 'meas'),
 ('dt_amb', 'amb'),
 ('dt_amb', 'sim'),
 ('meas', 'amb'),
 ('meas', 'sim'),
 ('amb', 'sim'),
 'amb_minus_meas',
 'meas_minus_amb',
 'abs_amb_meas_diff']

In [15]:
test_dict = {}
for item in test_set:

    if type(item) == tuple:
        key = "+".join(item)
        test_dict[key] = {}
        test_dict[key]["data"] =  [all_data[k] for k in item]
    else: # if not a tuple, then a string 
        test_dict[item] = {}
        test_dict[item]["data"] = [all_data[item]]
        

In [11]:
_meas = m.ML_Window_Detect([arrs["meas"]], window_sched)
_meas.run_all()

In [17]:
for k, v in test_dict.items():
    print(k)
    obj = m.ML_Window_Detect(v["data"], window_sched)
    obj.run_all()
    test_dict[k]["accuracy"] = obj.accuracy

dt_meas
dt_sim
dt_amb
dt_meas+dt_sim+dt_amb
meas
amb
sim
meas+amb+sim
dt_meas+dt_sim
dt_meas+dt_amb
dt_meas+meas
dt_meas+amb
dt_meas+sim
dt_sim+dt_amb
dt_sim+meas
dt_sim+amb
dt_sim+sim
dt_amb+meas
dt_amb+amb
dt_amb+sim
meas+amb
meas+sim
amb+sim
amb_minus_meas
meas_minus_amb
abs_amb_meas_diff


In [18]:
for k, v in test_dict.items():
    print(k, v["accuracy"])

dt_meas 0.6363636363636364
dt_sim 0.2805194805194805
dt_amb 0.4077922077922078
dt_meas+dt_sim+dt_amb 0.612987012987013
meas 0.5428571428571428
amb 0.5766233766233766
sim 0.44675324675324674
meas+amb+sim 0.5376623376623376
dt_meas+dt_sim 0.587012987012987
dt_meas+dt_amb 0.6441558441558441
dt_meas+meas 0.6753246753246753
dt_meas+amb 0.6103896103896104
dt_meas+sim 0.522077922077922
dt_sim+dt_amb 0.45714285714285713
dt_sim+meas 0.5168831168831168
dt_sim+amb 0.44415584415584414
dt_sim+sim 0.4103896103896104
dt_amb+meas 0.6493506493506493
dt_amb+amb 0.5636363636363636
dt_amb+sim 0.4805194805194805
meas+amb 0.6077922077922078
meas+sim 0.5376623376623376
amb+sim 0.574025974025974
amb_minus_meas 0.4909090909090909
meas_minus_amb 0.4909090909090909
abs_amb_meas_diff 0.4623376623376623


In [ ]:
_meas.accuracy

0.5428571428571428